# 我是進階的MongoDB 喔！

## The Aggregation Framework 
### Q: WHO TWEETED MOST:
1. Group tweets by user
2. Count each user's tweets
3. Select user with the most

* Sort into descending order 
* Select user at top

select user with the most = sort into descending order then select user at the top.
#### the content in aggregate() is called pipline.

In [ ]:
def most_tweets():
    result = db.tweets.aggregate([
        {"$group" : {"_id" : "$user.screen_name",  # _id is the key on different document # user is sub-document
                     "count" : {"$sum" : 1} } }, # sum by 1
        {"$sort" : {"count" : -1}}
    ])
    return result

# _id is the key means that new document has key=_id, value=content of field <.screen_name>
# ""$group" operator mean aggregate the same document by use.screen_name,
# $sum : sum operator
# "$" infornt of use.screen_name is not operator
# "$sort" means sorting the "count" number descendingly.

## Quiz

In [ ]:
#!/usr/bin/env python
"""
The tweets in our twitter collection have a field called "source". This field describes the application
that was used to create the tweet. Following the examples for using the $group operator, your task is 
to modify the 'make-pipeline' function to identify most used applications for creating tweets. 
As a check on your query, 'web' is listed as the most frequently used application.
'Ubertwitter' is the second most used. The number of counts should be stored in a field named 'count'
(see the assertion at the end of the script).

Please modify only the 'make_pipeline' function so that it creates and returns an aggregation pipeline
that can be passed to the MongoDB aggregate function. As in our examples in this lesson, the aggregation 
pipeline should be a list of one or more dictionary objects. 
Please review the lesson examples if you are unsure of the syntax.

Your code will be run against a MongoDB instance that we have provided. 
If you want to run this code locally on your machine, you have to install MongoDB, 
download and insert the dataset.
For instructions related to MongoDB setup and datasets please see Course Materials.

Please note that the dataset you are using here is a smaller version of the twitter dataset 
used in examples in this lesson. 
If you attempt some of the same queries that we looked at in the lesson examples,
your results will be different.
"""


def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [{"$group" : { "_id" : "$source",  # notice that add "$" before field. that means we tell the mongo read the value in source
                              "count" : {"$sum" : 1}}},
                 {"$sort" : {"count" : -1}}]
    return pipeline

def tweet_sources(db, pipeline):
    return [doc for doc in db.tweets.aggregate(pipeline)]

if __name__ == '__main__':
    db = get_db('twitter')
    pipeline = make_pipeline()
    result = tweet_sources(db, pipeline)
    import pprint
    pprint.pprint(result[0])
    assert result[0] == {u'count': 868, u'_id': u'web'}


## Aggregation Operation - An Overview
- \$project : only project some of field 
- \$match : match some document into aggregate
- \$group
- \$sum
- \$skip : skip three document, keep the firth document into aggregation
- \$limit : limit the three top document, also call the inversw skip
- \$unwind : if the field has an array with three values, and then unwund it for dividing three values into three different document with the same other contain.

### \$match and \$project and \$divide

In [ ]:
# Q: who has the hightest followers to friends ratio 

def highest_ration():
    result = db.tweets.aggregate([
        {"$macth" : {"user.friends_count" : {"$gt" : 0}}},
        {"$project" : {"ratio" : {"$divide" : ["$user.followers_count",  # $divide operator 除法, divide folloewers and friends 
                                               "$user.friends_count"]},
                      "screen_name" : "$user.screen_name"}}, 
        # the new stage has tow sub-document which is ratio and screen_name 
        {"$sort" : {"ratio" : -1} },
        {"$limit" : 1}
    ])

## Quiz

In [ ]:
#!/usr/bin/env python
"""
Write an aggregation query to answer this question:

Of the users in the "Brasilia" timezone who have tweeted 100 times or more,
who has the largest number of followers?

The following hints will help you solve this problem:
- Time zone is found in the "time_zone" field of the user object in each tweet.
- The number of tweets for each user is found in the "statuses_count" field.
  To access these fields you will need to use dot notation (from Lesson 4)
- Your aggregation query should return something like the following:
{u'ok': 1.0,
 u'result': [{u'_id': ObjectId('52fd2490bac3fa1975477702'),
                  u'followers': 2597,
                  u'screen_name': u'marbles',
                  u'tweets': 12334}]}
Note that you will need to create the fields 'followers', 'screen_name' and 'tweets'.

Please modify only the 'make_pipeline' function so that it creates and returns an aggregation 
pipeline that can be passed to the MongoDB aggregate function. As in our examples in this lesson,
the aggregation pipeline should be a list of one or more dictionary objects. 
Please review the lesson examples if you are unsure of the syntax.

Your code will be run against a MongoDB instance that we have provided. If you want to run this code
locally on your machine, you have to install MongoDB, download and insert the dataset.
For instructions related to MongoDB setup and datasets please see Course Materials.

Please note that the dataset you are using here is a smaller version of the twitter dataset used 
in examples in this lesson. If you attempt some of the same queries that we looked at in the lesson 
examples, your results will be different.
"""

def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [{"$match" : {"user.time_zone" : "Brasilia",  # Be careful the key, since "time_zone" is a sub-document under user
                             "user.statuses_count" : {"$gte" : 100} } },
                {"$project" : {"followers" :  "$user.followers_count",
                               "screen_name" : "$user.screen_name",
                               "tweets" : "$user.statuses_count"} },            
                {"$sort" : {"followers" : -1} },
                {"$limit" : 1}]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.tweets.aggregate(pipeline)]


if __name__ == '__main__':
    db = get_db('twitter')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    import pprint
    pprint.pprint(result)
    assert len(result) == 1
    assert result[0]["followers"] == 17209



# \$unwind
Who included the most user mentions?

In [ ]:
def user_mentions():
    results = db.tweets.aggregate([
        {"$unwind" : "$entities.user_mentions"},
        {"$group" : {"_id" : "$user.screen_name",
                     "count" : {"$sum" : 1} } },
        {"$sort" : {"count" : -1 } },
        {"$limit" : 1} ] )
    return result

# Quiz

In [ ]:
#!/usr/bin/env python
"""
For this exercise, let's return to our cities infobox dataset. The question we would like you to answer
is as follows:  Which region or district in India contains the most cities? (Make sure that the count of
cities is stored in a field named 'count'; see the assertions at the end of the script.)

As a starting point, use the solution for the example question we looked at -- "Who includes the most
user mentions in their tweets?"

One thing to note about the cities data is that the "isPartOf" field contains an array of regions or 
districts in which a given city is found. See the example document in Instructor Comments below.

Please modify only the 'make_pipeline' function so that it creates and returns an aggregation pipeline 
that can be passed to the MongoDB aggregate function. As in our examples in this lesson, the aggregation 
pipeline should be a list of one or more dictionary objects. Please review the lesson examples if you 
are unsure of the syntax.

Your code will be run against a MongoDB instance that we have provided. If you want to run this code 
locally on your machine, you have to install MongoDB, download and insert the dataset.
For instructions related to MongoDB setup and datasets please see Course Materials.

Please note that the dataset you are using here is a smaller version of the cities collection used in 
examples in this lesson. If you attempt some of the same queries that we looked at in the lesson 
examples, your results may be different.
"""

def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [{"$match" : {"country" : "India"}},
                {"$unwind" : "$isPartOf"},
                {"$group" : {"_id" : "$isPartOf",  #the "isPartOf" field contains an array of regions or districts 
                             "count" : {"$sum" : 1}}},
                {"$sort" : {"count" : -1 } },
                {"$limit" : 1}
                ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.cities.aggregate(pipeline)]

if __name__ == '__main__':
    db = get_db('examples')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    print "Printing the first result:"
    import pprint
    pprint.pprint(result[0])
    assert result[0]["_id"] == "Uttar Pradesh" # the answer is in "$isPartOf"
    assert result[0]["count"] == 623

## \$group operators
- \$sum
- \$frist
- \$last
- \$max
- \$min
- \$avg

In [ ]:
# average
def hashtag_retweet_avg():
    result = db.tweets.aggregate([
        {"$unwind" : "$entities.hashtags"},
        {"$group" : {"_id" : "$entities.hashtags.text",
                     "retweet_avg" : {"$avg" : "$retweet_count"}
                    } },
        {"$sort" : {"retweet_avg" : -1}}
    ])
    return result

### deal with array
- \$push : \$push is similar to $addToSet, it aggregates all values into an array
- \$addToSet :　把 array 重複數字忽略, 唯一的數字記下留在集合裡　(unique)

In [ ]:
def unique_hastags_by_user():
    results = db.tweets.aggregate([
        {"$unwind" : "$entitles.hashtags"},
        {"group" : {"_id" : "$user.screen_name",
                    "unique_hashtags" : {
                        "$addToSet" : "$entities.hashtags.text"
                    } } },
        {"$sort" : {"_id": -1}}
    ])
    
    return result

## Quiz

In [ ]:
#!/usr/bin/env python
"""
$push is similar to $addToSet. The difference is that rather than accumulating only unique values 
it aggregates all values into an array.

Using an aggregation query, count the number of tweets for each user. In the same $group stage, 
use $push to accumulate all the tweet texts for each user. Limit your output to the 5 users
with the most tweets. 
Your result documents should include only the fields:
"_id" (screen name of user), 
"count" (number of tweets found for the user),
"tweet_texts" (a list of the tweet texts found for the user).  

Please modify only the 'make_pipeline' function so that it creates and returns an aggregation 
pipeline that can be passed to the MongoDB aggregate function. As in our examples in this lesson, 
the aggregation pipeline should be a list of one or more dictionary objects. 
Please review the lesson examples if you are unsure of the syntax.

Your code will be run against a MongoDB instance that we have provided. If you want to run this code 
locally on your machine, you have to install MongoDB, download and insert the dataset.
For instructions related to MongoDB setup and datasets please see Course Materials.

Please note that the dataset you are using here is a smaller version of the twitter dataset used in 
examples in this lesson. If you attempt some of the same queries that we looked at in the lesson 
examples, your results will be different.
"""

def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [ 
        {"$group" : {"_id" : "$user.screen_name", 
                     "count" : {"$sum" : 1},  # count how many texts
                     "tweet_texts" : {
                         "$push" : "$text" # show the content of texts
                     } } },
        {"$sort" : {"count" : -1}},
        {"$limit" : 5}
        ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.twitter.aggregate(pipeline)]


if __name__ == '__main__':
    db = get_db('twitter')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    import pprint
    pprint.pprint(result)
    assert len(result) == 5
    assert result[0]["count"] > result[4]["count"]
    sample_tweet_text = u'Take my money! #liesguystell http://movie.sras2.ayorganes.com'
    assert result[4]["tweet_texts"][0] == sample_tweet_text
    

## Multiple group stage

In [ ]:
def user_mention():
    result = db.tweets.aggregate([
        {"$unwind" : "$entities.user_mentions"},
        {"group": {
            "_id" : "$user.screen_name",
            "mest" : {
                "$addToSet" : "$entities.user_mentions.screen_name"
            } } },
        {"$unwind" : "$mset"},
        {"$group" : {"_id" : "$_id", "count" : {"$sum" : 1}}}
        {"$sort" : {"count" : -1 } },
        {"$limit" : 10}
    ])
    return result

In [5]:
abs(93599.7764084507 - 201128.0241546919) <  10 ** -8

False